# Exercicio 01 - Coleta, Preparação e Análise de Dados

### Grupo: Giovani Cancherini
### Data de Entrega: 07/04/2024 (Turma 10)

---

## Tarefa 1 – Desenvolvendo um crawler

Escreva um crawler para descobrir e coletar páginas da Wikipedia em Português. Seu programa deve coletar 5.000 páginas de verbetes diferentes da Wikipedia, à partir da página
inicial: https://pt.wikipedia.org. Não coletar páginas de outros sites e também não deve
ser coletado páginas internas da Wikipedia que não são verbetes.

O crawler deve funcionar da seguinte maneira:
1. Obter uma página.
2. Salvar a página como um arquivo html, chamado <titulo_verbete>.html
3. Extrair todos os links que se encontram nessa página
4. Filtrar os links, removendo os que não se referem à verbetes e os verbetes que já foram
visitados.
1
5. Guardar esses links em uma lista
6. Escolher um link não visitado para ser a próxima página.
7. Voltar ao passo inicial
As páginas de verbetes coletadas deverão ser salvas como arquivos com a extensão .html.
Lembre-se de tomar cuidado para não estressar o servidor com requisições em excesso

In [29]:
# import requests
import requests
import time 

# BeautifulSoup permite navegar no html
from bs4 import BeautifulSoup

In [30]:
def salvar_links(soup):
    links_obtidos = []
    todos_links = soup.find(id="bodyContent").find_all("a")
    
    for link in todos_links:
        if "href" in link.attrs.keys() and link["href"].startswith("/wiki/"):
            if link["href"].find(":") == -1:
                links_obtidos.append(link["href"])
                # print(links_obtidos[-1])
                
    return links_obtidos

In [31]:
def salvar_html(nome_arquivo, response):
    subpath = "Exercicio1-paginas/"
    fullpath = f"{subpath}{nome_arquivo}.html"
    with open(fullpath, "w", encoding="utf-8") as f:
        conteudo = response.content.decode("utf-8")
        f.write(conteudo)

In [32]:
lista_paginas_visitadas = []
lista_paginas_nao_visitadas = []
url_base = "https://pt.wikipedia.org"

def obtem_pagina_recursivamente(url, contador_repeticoes):
    if contador_repeticoes == 0:
        print()
        print()
        print("[SUCESSO]")
        print("Todas as paginas obtidas!")
        return
    
    time.sleep(1)
    
    response = requests.get(url)
    lista_paginas_visitadas.append(url)
    
    soup = BeautifulSoup(response.content, "html.parser")
    titulo = soup.select(".mw-page-title-main")
    nome_arquivo = titulo[0].text.strip().replace(" ", "_") if titulo else "pagina_sem_titulo"

    links_obtidos = salvar_links(soup)
    salvar_html(nome_arquivo, response)
    
    for link in links_obtidos:
        link_completo = url_base + link
        if link_completo not in lista_paginas_visitadas and link_completo not in lista_paginas_nao_visitadas:
            lista_paginas_nao_visitadas.append(link_completo)
       
    if lista_paginas_nao_visitadas:
        lista_paginas_nao_visitadas.pop(0)
    else:
        print()
        print()
        print("[ATENCAO]")
        print("Nao ha mais paginas para visitar!")
        
        
    if lista_paginas_nao_visitadas:
        return obtem_pagina_recursivamente(lista_paginas_nao_visitadas[0], contador_repeticoes - 1)
    else:
        print()
        print()
        print("[ATENCAO]")
        print("Nao ha mais paginas para visitar!")
        return

In [33]:
# Inicia a busca
## Verificar output em /Exercicio1-paginas/

pagina_de_inicio = "https://pt.wikipedia.org"
numero_de_paginas = 50 # para teste final = 5000

obtem_pagina_recursivamente(pagina_de_inicio, numero_de_paginas)



[SUCESSO]
Todas as paginas obtidas!


---

## Tarefa 2 – Extraindo informações de Infoboxes

A segunda tarefa consiste em identificar as páginas que possuem infoboxes, que são usadas
para resumir as informações de um artigo na Wikipédia. Veja na figura a seguir um exemplo
de página que contém um infobox (ele está destacado em vermelho na imagem).
Infoboxes estruturam informações de diversas maneiras, por isso é difícil conseguir extrair
todos os seus elementos de forma fácil. Portanto, focaremos nossos esforços em extrair apenas
alguns elementos. São eles:
1. Título: toda infobox possui um título que fica no topo da caixa. No exemplo da figura,
o título é Alan Turing
2. Pares chave – valor: esses pares são identificados por uma chave que está associada a
um único valor. Por exemplo, na figura temos a chave “Nome completo” e o valor “Alan
Mathison Turing”.
3. Pares chave – lista: nesse tipo de item, uma chave está associada a uma lista de valores.
Na figura de exemplo são pares de chave – lista os campos: Conhecido(a) por, Alma
mater, Orientado(a)(s), Instituições e Campos.
Sua tarefa consiste em extrair o conteúdo das infoboxes de todas as páginas que foram extraídas no exercício anterior. Seu programa deve identificar quando uma página possui uma
infobox, realizar a extração das informações e salvá-las em um arquivo .json cujo nome é o
título da infobox.
Para fins de teste, será fornecido um conjunto de páginas juntamente com a saída esperada
para cada uma delas.

Sua tarefa consiste em extrair o conteúdo das infoboxes de todas as páginas que foram extraídas no exercício anterior. Seu programa deve identificar quando uma página possui uma
infobox, realizar a extração das informações e salvá-las em um arquivo .json cujo nome é o
título da infobox.
Para fins de teste, será fornecido um conjunto de páginas juntamente com a saída esperada
para cada uma delas.
